In [1]:
import os
os.getcwd()


'/Users/baburambista/Desktop/face-verification/face-datasets'

In [37]:
import pandas as pd
import tensorflow as tf

IMG_SIZE = 160
ROOT = "/Users/baburambista/Desktop/face-verification/face-datasets/"

df = pd.read_csv(ROOT + "face_identification.csv")

def load_image(path):
    # path = ROOT + path
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    return img / 255.0
    

In [38]:
def data_generator(df): #loads image row by row
    for _, row in df.iterrows():       #_ is the throwuse field(index, row) as iterrow gives us that two values
        img1 = load_image(ROOT + row["image_1"]) #It looks up the value for the column "image_2".
        img2 = load_image(ROOT + row["image_2"])
        label = float(row["target"])
        yield (img1, img2), label #The generator produces one element at a time.Each element is a tuple of two things: img and label

##creates the dataset that Batches them into groups here 8, images at a time Prefetches them for efficient GPU/CPU training
dataset = tf.data.Dataset.from_generator( #tf.data like a folder of tools for loading, transforming, and feeding data efficiently. Dataset is a class inside tf.data from which we can iterate over, batch, shuffle, and feed to a model.
    lambda: data_generator(df), #lamba returns no arguments 
    output_signature=( #defines the shape , type of the data
        (
            tf.TensorSpec((IMG_SIZE,IMG_SIZE,3), tf.float32), #yield value is used here ((img1_tensor, img2_tensor), 1)
            tf.TensorSpec((IMG_SIZE,IMG_SIZE,3), tf.float32),
        ),
        tf.TensorSpec((), tf.int32)
    )
)

dataset = dataset.shuffle(1024).batch(8).repeat().prefetch(tf.data.AUTOTUNE) #we combine the 8 row in one for training (8, 412, 412, 3)

steps_per_epoch = len(df) 

In [39]:
#manually trained model
# from tensorflow.keras import layers, Model

# def build_encoder():
#     inp = layers.Input((IMG_SIZE,IMG_SIZE,3))

#     x = layers.Conv2D(32, 3, activation='relu')(inp)
#     x = layers.MaxPool2D()(x)

#     x = layers.Conv2D(64, 3, activation='relu')(x)
#     x = layers.MaxPool2D()(x)

#     x = layers.Conv2D(128, 3, activation='relu')(x)
#     x = layers.GlobalAveragePooling2D()(x)

#     x = layers.Dense(128)(x)  # embedding vector
#     x = layers.Lambda(lambda t: tf.math.l2_normalize(t, axis=1))(x)

#     return Model(inp, x)

# encoder = build_encoder()
# encoder.summary()

#using the pretraied model 
from tensorflow.keras.applications import MobileNetV2

def build_encoder():
    inp = layers.Input((IMG_SIZE, IMG_SIZE, 3))
    
    # Pretrained MobileNetV2, exclude top layers
    base_model = MobileNetV2(input_tensor=inp, include_top=False, weights='imagenet')
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128)(x)  # embedding vector
    x = layers.Lambda(lambda t: tf.math.l2_normalize(t, axis=1))(x)  # L2 normalize

    return Model(inp, x)


In [40]:
img1 = layers.Input((IMG_SIZE,IMG_SIZE,3))
img2 = layers.Input((IMG_SIZE,IMG_SIZE,3))

e1 = encoder(img1)
e2 = encoder(img2)

# Euclidean distance
distance = layers.Lambda(lambda tensors: tf.norm(tensors[0] - tensors[1], axis=1, keepdims=True))([e1, e2])
model = Model([img1, img2], distance)

def contrastive_loss(y_true, d, margin=1):
    y_true = tf.cast(y_true, tf.float32)
    return tf.reduce_mean(
        y_true * tf.square(d) + 
        (1 - y_true) * tf.square(tf.maximum(margin - d, 0))
    )


model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=lambda y, d: contrastive_loss(y, d, margin=1)  # margin=1 is standard
)

steps_per_epoch = len(df) // 8

model.fit(dataset, epochs=5, steps_per_epoch=steps_per_epoch)


Epoch 1/5
832/832 ━━━━━━━━━━━━━━━━━━━━ 54s 64ms/step - loss: 0.1118    
Epoch 2/5
832/832 ━━━━━━━━━━━━━━━━━━━━ 56s 67ms/step - loss: 0.1372 
Epoch 3/5
832/832 ━━━━━━━━━━━━━━━━━━━━ 58s 70ms/step - loss: 0.1327 
Epoch 4/5
832/832 ━━━━━━━━━━━━━━━━━━━━ 59s 71ms/step - loss: 0.1249 
Epoch 5/5
832/832 ━━━━━━━━━━━━━━━━━━━━ 63s 76ms/step - loss: 0.1182 


In [41]:
img1 = load_image(ROOT + "faces/Alexis Bledel/cropped_Alexis Bledel_4.jpg")
img2 = load_image(ROOT + "faces/Alexis Bledel/cropped_Alexis Bledel_10.jpg")

img1 = tf.expand_dims(img1, axis=0)
img2 = tf.expand_dims(img2, axis=0)

dist = model.predict([img1, img2])[0][0]
print("Distance:", dist)


# ✅ Threshold check
THRESHOLD = 0.5
same = dist < THRESHOLD
print("Same person?", same)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Distance: 1.3585352
Same person? False


In [42]:
img1 = load_image(ROOT + "faces/Alexis Bledel/cropped_Alexis Bledel_4.jpg")
img2 = load_image(ROOT + "faces/Adam Sandler/cropped_Adam Sandler_8.jpg")

img1 = tf.expand_dims(img1, axis=0)  # (1, 160, 160, 3)
img2 = tf.expand_dims(img2, axis=0)

dist = model.predict([img1, img2])[0][0]
print("Distance:", dist)

THRESHOLD = 0.5
same = dist < THRESHOLD
print("Same person?", same)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Distance: 1.318017
Same person? False
